# NAVER BLOG Crawling

- impot

In [5]:
import os
import sys
import pandas as pd
import time
from datetime import datetime, date, timedelta
from bs4 import BeautifulSoup

from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from selenium.webdriver.chrome.options import Options


from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import warnings
warnings.filterwarnings('ignore')


- title/url 

In [4]:
# 검색어 입력
query = input("검색어 : ")


# 조회 기간 설정 : 7일(days에서 수정 가능)
# startDate=7일전 날짜, endDate=오늘 날짜
date = datetime.now()
startDate= (date+timedelta(days=-7)).strftime('%Y-%m-%d')
endDate = (datetime.now()).strftime('%Y-%m-%d')

print(startDate, endDate)


# Chrome driver 환경설정 및 실행
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_argument('headless')
driver = webdriver.Chrome(options = options)
base_url = f"https://section.blog.naver.com/Search/Post.naver?pageNo=1&rangeType=WEEK&orderBy=sim&startDate={startDate}&endDate={endDate}&keyword={query}"
driver.get(base_url)
time.sleep(1)


# 총 검색 결과
html = driver.page_source
soup = BeautifulSoup(html, "html.parser")
search_number = soup.find(class_="search_number")
print("총 블로그글 개수 : ", search_number.text)


# 연관 검색어 
sub_keywords_data = soup.select('div.area_keyword > div.list')
sub_keywords = []
for i in sub_keywords_data: 
    sub_keywords.append(i.get_text())

print("연관검색어:",*sub_keywords)


# blog url/title 가져오기
total_num = search_number.text.replace("건",'').replace(",",'')
total_num = int(total_num)
page_num = 1
end_page=total_num//7+1
end_page
contents_num = 7

blog_title_lst = []
blog_url_lst = []

while contents_num == 7 : 
    search_url = f"https://section.blog.naver.com/Search/Post.naver?pageNo={page_num}&rangeType=WEEK&orderBy=sim&startDate={startDate}&endDate={endDate}&keyword={query}"
    driver.get(search_url)
    time.sleep(0.5)
    
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    
    posts_data = soup.select('div.info_post > div.desc > a.desc_inner')
    
    # 블로그 제목, url 가져오기
    posts_data = soup.select('div.info_post > div.desc > a.desc_inner')

    for post in posts_data :
        title=post.get_text().replace('\n','').strip()
        href=post.attrs['href']
        blog_title_lst.append(title)
        blog_url_lst.append(href)
    
    page_num +=1
    contents_num = len(posts_data)

print(blog_title_lst)
print(blog_url_lst)

2024-05-01 2024-05-08
총 블로그글 개수 :  1,123건
연관검색어: 
구로디지털단지역 술집구로디지털단지역 카페구로디지털단지역 미용실구로디지털단지역 피부과구로디지털단지역 오피스텔구로디지털단지역 네일구로디지털단지역 아파트구로디지털단지역 별밤구로디지털단지역 공영주차장구로디지털단지역 주차장구로디지털단지역 다이소구로디지털단지역 놀거리구로디지털단지역 고기집구로디지털단지역 스타벅스구로디지털단지역 호텔구로디지털단지역 영화관구로디지털단지역 빵집구로디지털단지역 이마트구로디지털단지역 날씨구로디지털단지역 6번출구


['구로디지털단지역 맛집 프리미엄 부위가 일품 [참숯구이 육향]', '구로디지털단지역한의원 또는 양천구병원에서 입원 재활치료', '구로디지털단지역 고기집 야자숯에 구워 풍부한 육즙이 느껴진 인생맛껍 구로본점', '구로디지털단지역 맛집 가성비 소고기 세트가 있는 조선화로집 구로디지털점', '구로디지털단지역안경 으뜸50안경 구로디지털단지역점 여행대비 선글라스', '[구로디지털단지역 맛집] 특색있는 프리미엄 양갈비 맛집, 양원집 구로디지털단지점!', '구로디지털단지역 깔깔거리 구디맛집 경성양꼬치 :: 입에서 살살 녹는 양갈비 양꼬치...', '구로디지털단지역술집맛집 대두족발', '구로디지털단지역 PT 여성전문헬스장 휘트니스피플우먼 구로디지털단지점 PT 체험...', '구로디지털단지역피부과 얼굴 선이 갸름하게', '[구디맛집] 대창 가득 구디 밥집 5.13 낙곱새 구로디지털단지역', '구로디지털단지역 반응 좋은 조개찜 맛집 물찬조개', '구로디지털단지역미용실 펌 후기 l 중단발 레이어드 C컬펌', '구디고깃집 / 형제특수부위 고기가 맛있는 구로디지털단지역 맛집', '구로디지털단지역 생활권 관악구 신림동 신축빌라 분양 매매', '구로디지털단지역 점심 고기 맛집 의영이네 양갈비', '구로동 지식산업센터 구로디지털단지역 마리오타워 사무실 임대', '[구로디지털단지역 맛집] 경성양꼬치', '구디 브레드밀레 연유빵 먹으러 구로디지털단지역 빵집까지 다녀온 후기', '

In [7]:
# 확인
print(len(blog_title_lst),len(blog_url_lst))

1123 1123


In [8]:
df = pd.DataFrame({"title":blog_title_lst, "url":blog_url_lst})
df.info()
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1123 entries, 0 to 1122
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   1123 non-null   object
 1   url     1123 non-null   object
dtypes: object(2)
memory usage: 17.7+ KB


,title,url
0,구로디지털단지역 맛집 프리미엄 부위가 일품 [참숯구이 육향],https://blog.naver.com/agness888/223439262998
1,구로디지털단지역한의원 또는 양천구병원에서 입원 재활치료,https://blog.naver.com/lavish001/223440020562
2,구로디지털단지역 고기집 야자숯에 구워 풍부한 육즙이 느껴진 인생맛껍 구로본점,https://blog.naver.com/jang9369/223439007190
3,구로디지털단지역 맛집 가성비 소고기 세트가 있는 조선화로집 구로디지털점,https://blog.naver.com/v8v8v8v8v8/223433538129
4,구로디지털단지역안경 으뜸50안경 구로디지털단지역점 여행대비 선글라스,https://blog.naver.com/hwaja75/223439118690
...,...,...
1118,가산고기집 대식당 쫄깃한 식감과 풍부한 육즙을 가진 어디서도 맛보지 못한 특별한...,https://blog.naver.com/kamilra_24/223433091323
1119,회식맛집 구디맛집 목구멍 구로디지털단지점,https://blog.naver.com/bbobbo0831/223433105809
1120,붉은눈썹문신 제거 잘하는 미작뷰티 구로점 후기,https://blog.naver.com/y_5117/223433109304
1121,구디고기맛집 구디맛집 남영동양문 구로디지털단지점,https://blog.naver.com/feguri/223439259550


- blog 본문 가져오기(test)

In [14]:
# Chrome driver 실행
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_argument('headless')
driver = webdriver.Chrome(options = options)


# 해당 blog 불러오기 
url = blog_url_lst[0]
driver.get(url)
time.sleep(1)


# iframe 접근 
driver.switch_to.frame('mainFrame')


# 제목 가져오기
overlays_title = ".se-module.se-module-text.se-title-text"
html_title = driver.find_element(By.CSS_SELECTOR, overlays_title)
title = html_title.text
print(title)


# 포스팅 날짜 가져오기
overlays_publishDate = ".se_publishDate.pcol2"
html_publishDate = driver.find_element(By.CSS_SELECTOR, overlays_publishDate)
date = html_publishDate.text
print(date)

# 본문 가져오기 
overlays_content = ".se-component.se-text.se-l-default"
html_content = driver.find_elements(By.CSS_SELECTOR, overlays_content)
contents_lst = []
for content in html_content : 
    contents_lst.append(content.text.replace('\n',''))
    

print(contents_lst)

# 해시태그 가져오기 

# title = html_title.text 
# title





구로디지털단지역 맛집 프리미엄 부위가 일품 [참숯구이 육향]
19시간 전
['누구라도 특별한 날이 다가오면뭐 할지 고민할 것 같은데요.특히나 어른을 모시고 어딘가에가야 하는 경우 더 그렇지 않나 싶어요.부모님과 친한 편인데도 불구하고어버이날 무엇을 준비하면 좋을지쉽사리 결정 내리지 못했어요!다행히 이번엔 구로디지털단지역 맛집에서제 주도 하에 가족이 모여 식사를 했는데맛도 그렇고 친절함 등 전반적으로만족스러워서 뿌듯한 미리 어버이날기념으로 먹고왔어요!', '저희와 같은 생각인지 많이들 외식을하는 날이라 저는 미리 예약을 했는데신의 한 수였어요.평이 좋은 여느 음식점이 보통 그런데여기도 식사 시간대에는 손님이꽤 몰리는 모습이었거든요.', '그래도 복잡한 분위기는 아니라부모님 모시고 가기에 무리는없었어요', '메뉴도 프리미엄 등급 토마호크와한돈 꽃목살, 한돈 숄더랙, 그리고우대갈비까지 아무 곳에서나맛볼 수 없는 귀한 부위 위주로준비되어 있으니 특별한 날에방문하기 좋은 곳이었어요.', '메뉴는 부모님이 드시고 싶어하시는걸로 선택을 해서 주문했어요!결론적으로는 잘했다고 봅니다.쉽게 접하기 어려운 부위이기도 하고맛도 굉장히 훌륭했으니까요.그렇게 주문하고 기다리다 보니기본 반찬이 테이블 위로하나 둘 올라오기 시작했어요.이건 무를 적당히 얇게 썰어서양념한 건데 고기를 먹다 보면 어느새슬쩍 스며드는 느끼함을 잡아줄 때큰 역할을 해주었습니다.', '', '젓갈은 밥이랑 먹을 때도 맛있는데의외로 고기랑도 잘 어울리더라고요!식당에서 식사할 때마다 젓갈을 주면그렇게 반가운데요. 이날도마찬가지였어요. 쌈을 싸먹을 때도잘 어울린답니다.약간 흐물거리는 게 나오는 곳도있는데 구로디지털단지역 맛집은기본찬 수준도 훌륭해서 쫄깃하고감칠맛까지 났어요.', '고기 좀 좋아한다, 먹을 줄 안다~이런 분이면 아실 거에요.숯이 얼마나 중요한지를 말이에요.좋지 않은 걸 쓰면 일단 건강에도좋지 않고 향이 베이지 않아 불맛을입히는 데에도 지장이 생기는불상사가 생기죠.그래서 항상 어떤 숯이 나오는지신경 써서 살

- blog 본문 크롤링

In [9]:
# Chrome driver 실행
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_argument('headless')
driver = webdriver.Chrome()

crawling_blog_data = []


# blog 본문 가져오기 
for i in range(0, 3) :    
    url = str(blog_url_lst[i])
    driver.get(url)
    time.sleep(1)
    
    
    # 본문 크롤링 예외 처리 
    try :
        # iframe 접근 
        driver.switch_to.frame('mainFrame')

        
        # 제목 가져오기
        overlays_title = ".se-module.se-module-text.se-title-text"
        html_title = driver.find_element(By.CSS_SELECTOR, overlays_title)
        title = html_title.text

        
        # 작성 날짜 가져오기
        overlays_publishDate = ".se_publishDate.pcol2"
        html_publishDate = driver.find_element(By.CSS_SELECTOR, overlays_publishDate)
        date = html_publishDate.text

        
        # 본문 가져오기
        overlays_content = ".se-component.se-text.se-l-default"
        html_content = driver.find_elements(By.CSS_SELECTOR, overlays_content)
        contents_lst = []
        for content in html_content : 
            contents_lst.append(content.text.replace('\n',''))
        content_str = ''.join(contents_lst)

        
        # 크롤링한 내용 담기 
        crawling_blog_data.append({"title":title, "date":date, "content":content_str})
        
        # driver 종료 
        driver.close
        time.sleep(1)
        
        
    # 에러 발생시 다음 블로그로 이동
    except : 
        print("error",i,title)
        driver.close()
        time.sleep(1)


In [10]:
print(crawling_blog_data)

[{'title': '구로디지털단지역 맛집 프리미엄 부위가 일품 [참숯구이 육향]', 'date': '20시간 전', 'content': '누구라도 특별한 날이 다가오면뭐 할지 고민할 것 같은데요.특히나 어른을 모시고 어딘가에가야 하는 경우 더 그렇지 않나 싶어요.부모님과 친한 편인데도 불구하고어버이날 무엇을 준비하면 좋을지쉽사리 결정 내리지 못했어요!다행히 이번엔 구로디지털단지역 맛집에서제 주도 하에 가족이 모여 식사를 했는데맛도 그렇고 친절함 등 전반적으로만족스러워서 뿌듯한 미리 어버이날기념으로 먹고왔어요!저희와 같은 생각인지 많이들 외식을하는 날이라 저는 미리 예약을 했는데신의 한 수였어요.평이 좋은 여느 음식점이 보통 그런데여기도 식사 시간대에는 손님이꽤 몰리는 모습이었거든요.그래도 복잡한 분위기는 아니라부모님 모시고 가기에 무리는없었어요메뉴도 프리미엄 등급 토마호크와한돈 꽃목살, 한돈 숄더랙, 그리고우대갈비까지 아무 곳에서나맛볼 수 없는 귀한 부위 위주로준비되어 있으니 특별한 날에방문하기 좋은 곳이었어요.메뉴는 부모님이 드시고 싶어하시는걸로 선택을 해서 주문했어요!결론적으로는 잘했다고 봅니다.쉽게 접하기 어려운 부위이기도 하고맛도 굉장히 훌륭했으니까요.그렇게 주문하고 기다리다 보니기본 반찬이 테이블 위로하나 둘 올라오기 시작했어요.이건 무를 적당히 얇게 썰어서양념한 건데 고기를 먹다 보면 어느새슬쩍 스며드는 느끼함을 잡아줄 때큰 역할을 해주었습니다.젓갈은 밥이랑 먹을 때도 맛있는데의외로 고기랑도 잘 어울리더라고요!식당에서 식사할 때마다 젓갈을 주면그렇게 반가운데요. 이날도마찬가지였어요. 쌈을 싸먹을 때도잘 어울린답니다.약간 흐물거리는 게 나오는 곳도있는데 구로디지털단지역 맛집은기본찬 수준도 훌륭해서 쫄깃하고감칠맛까지 났어요.고기 좀 좋아한다, 먹을 줄 안다~이런 분이면 아실 거에요.숯이 얼마나 중요한지를 말이에요.좋지 않은 걸 쓰면 일단 건강에도좋지 않고 향이 베이지 않아 불맛을입히는 데에도 지장이 생기는불상사가 생기죠.그래서 항상 어떤 숯이 나오는지

# preprocessing

- DataFrame

In [27]:
import numpy as np
import pandas as pd
import re
import os
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
from konlpy.tag import Twitter

In [32]:
df = pd.DataFrame(crawling_blog_data)
df

,title,date,content
0,구로디지털단지역 맛집 프리미엄 부위가 일품 [참숯구이 육향],20시간 전,누구라도 특별한 날이 다가오면뭐 할지 고민할 것 같은데요.특히나 어른을 모시고 어딘...
1,구로디지털단지역한의원 또는 양천구병원에서 입원 재활치료,3시간 전,"평소 매일 이용하는 도로라고 해도 불쑥 튀어나올 수 있는 모든 차량, 보행자를 주의..."
2,구로디지털단지역 고기집 야자숯에 구워 풍부한 육즙이 느껴진 인생맛껍 구로본점,2024. 5. 7. 15:38,위치 : 서울 구로구 디지털로34길 55 B107호영업시간 : 12:00 ~ 23:...


In [38]:
content_data = [content for content in df['content']]
content_data

['누구라도 특별한 날이 다가오면뭐 할지 고민할 것 같은데요.특히나 어른을 모시고 어딘가에가야 하는 경우 더 그렇지 않나 싶어요.부모님과 친한 편인데도 불구하고어버이날 무엇을 준비하면 좋을지쉽사리 결정 내리지 못했어요!다행히 이번엔 구로디지털단지역 맛집에서제 주도 하에 가족이 모여 식사를 했는데맛도 그렇고 친절함 등 전반적으로만족스러워서 뿌듯한 미리 어버이날기념으로 먹고왔어요!저희와 같은 생각인지 많이들 외식을하는 날이라 저는 미리 예약을 했는데신의 한 수였어요.평이 좋은 여느 음식점이 보통 그런데여기도 식사 시간대에는 손님이꽤 몰리는 모습이었거든요.그래도 복잡한 분위기는 아니라부모님 모시고 가기에 무리는없었어요메뉴도 프리미엄 등급 토마호크와한돈 꽃목살, 한돈 숄더랙, 그리고우대갈비까지 아무 곳에서나맛볼 수 없는 귀한 부위 위주로준비되어 있으니 특별한 날에방문하기 좋은 곳이었어요.메뉴는 부모님이 드시고 싶어하시는걸로 선택을 해서 주문했어요!결론적으로는 잘했다고 봅니다.쉽게 접하기 어려운 부위이기도 하고맛도 굉장히 훌륭했으니까요.그렇게 주문하고 기다리다 보니기본 반찬이 테이블 위로하나 둘 올라오기 시작했어요.이건 무를 적당히 얇게 썰어서양념한 건데 고기를 먹다 보면 어느새슬쩍 스며드는 느끼함을 잡아줄 때큰 역할을 해주었습니다.젓갈은 밥이랑 먹을 때도 맛있는데의외로 고기랑도 잘 어울리더라고요!식당에서 식사할 때마다 젓갈을 주면그렇게 반가운데요. 이날도마찬가지였어요. 쌈을 싸먹을 때도잘 어울린답니다.약간 흐물거리는 게 나오는 곳도있는데 구로디지털단지역 맛집은기본찬 수준도 훌륭해서 쫄깃하고감칠맛까지 났어요.고기 좀 좋아한다, 먹을 줄 안다~이런 분이면 아실 거에요.숯이 얼마나 중요한지를 말이에요.좋지 않은 걸 쓰면 일단 건강에도좋지 않고 향이 베이지 않아 불맛을입히는 데에도 지장이 생기는불상사가 생기죠.그래서 항상 어떤 숯이 나오는지신경 써서 살펴보는데요.여긴 이렇게 참숯이 나와서 먹기전부터 기대가 됐어요주문한 고기가 나왔는데 육질이좋은게 느껴지는게 선홍빛이선명하게 

In [42]:
clensing_content_data = []
for content in content_data : 
    clensing_data = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]','',content)
    clensing_content_data.append(clensing_data)

print(clensing_content_data)

['누구라도 특별한 날이 다가오면뭐 할지 고민할 것 같은데요특히나 어른을 모시고 어딘가에가야 하는 경우 더 그렇지 않나 싶어요부모님과 친한 편인데도 불구하고어버이날 무엇을 준비하면 좋을지쉽사리 결정 내리지 못했어요다행히 이번엔 구로디지털단지역 맛집에서제 주도 하에 가족이 모여 식사를 했는데맛도 그렇고 친절함 등 전반적으로만족스러워서 뿌듯한 미리 어버이날기념으로 먹고왔어요저희와 같은 생각인지 많이들 외식을하는 날이라 저는 미리 예약을 했는데신의 한 수였어요평이 좋은 여느 음식점이 보통 그런데여기도 식사 시간대에는 손님이꽤 몰리는 모습이었거든요그래도 복잡한 분위기는 아니라부모님 모시고 가기에 무리는없었어요메뉴도 프리미엄 등급 토마호크와한돈 꽃목살 한돈 숄더랙 그리고우대갈비까지 아무 곳에서나맛볼 수 없는 귀한 부위 위주로준비되어 있으니 특별한 날에방문하기 좋은 곳이었어요메뉴는 부모님이 드시고 싶어하시는걸로 선택을 해서 주문했어요결론적으로는 잘했다고 봅니다쉽게 접하기 어려운 부위이기도 하고맛도 굉장히 훌륭했으니까요그렇게 주문하고 기다리다 보니기본 반찬이 테이블 위로하나 둘 올라오기 시작했어요이건 무를 적당히 얇게 썰어서양념한 건데 고기를 먹다 보면 어느새슬쩍 스며드는 느끼함을 잡아줄 때큰 역할을 해주었습니다젓갈은 밥이랑 먹을 때도 맛있는데의외로 고기랑도 잘 어울리더라고요식당에서 식사할 때마다 젓갈을 주면그렇게 반가운데요 이날도마찬가지였어요 쌈을 싸먹을 때도잘 어울린답니다약간 흐물거리는 게 나오는 곳도있는데 구로디지털단지역 맛집은기본찬 수준도 훌륭해서 쫄깃하고감칠맛까지 났어요고기 좀 좋아한다 먹을 줄 안다이런 분이면 아실 거에요숯이 얼마나 중요한지를 말이에요좋지 않은 걸 쓰면 일단 건강에도좋지 않고 향이 베이지 않아 불맛을입히는 데에도 지장이 생기는불상사가 생기죠그래서 항상 어떤 숯이 나오는지신경 써서 살펴보는데요여긴 이렇게 참숯이 나와서 먹기전부터 기대가 됐어요주문한 고기가 나왔는데 육질이좋은게 느껴지는게 선홍빛이선명하게 보이고 마블링도 너무과하지 않게 있었어요직원분

In [55]:
content_words = []
twitter = Twitter()

for words in clensing_content_data : 
    word = twitter.pos(words)
    content_words.append(word)

print(len(content_words[0]))    
content_words[0]

1036


[('누구', 'Noun'),
 ('라도', 'Josa'),
 ('특별한', 'Adjective'),
 ('날', 'Noun'),
 ('이', 'Josa'),
 ('다가오면', 'Verb'),
 ('뭐', 'Noun'),
 ('할지', 'Verb'),
 ('고민', 'Noun'),
 ('할', 'Verb'),
 ('것', 'Noun'),
 ('같은데요', 'Adjective'),
 ('특히', 'Adverb'),
 ('나', 'Noun'),
 ('어른', 'Noun'),
 ('을', 'Josa'),
 ('모시', 'Noun'),
 ('고', 'Josa'),
 ('어딘가', 'Noun'),
 ('에', 'Josa'),
 ('가야', 'Noun'),
 ('하는', 'Verb'),
 ('경우', 'Noun'),
 ('더', 'Noun'),
 ('그렇지', 'Adjective'),
 ('않나', 'Verb'),
 ('싶어요', 'Verb'),
 ('부모님', 'Noun'),
 ('과', 'Josa'),
 ('친한', 'Adjective'),
 ('편인데도', 'Verb'),
 ('불구', 'Noun'),
 ('하고', 'Verb'),
 ('어버이날', 'Noun'),
 ('무엇', 'Noun'),
 ('을', 'Josa'),
 ('준비', 'Noun'),
 ('하면', 'Verb'),
 ('좋을지', 'Adjective'),
 ('쉽사리', 'Noun'),
 ('결정', 'Noun'),
 ('내', 'Determiner'),
 ('리지', 'Noun'),
 ('못', 'VerbPrefix'),
 ('했어요', 'Verb'),
 ('다행히', 'Adjective'),
 ('이번', 'Noun'),
 ('엔', 'Josa'),
 ('구로디지털단지역', 'Noun'),
 ('맛집', 'Noun'),
 ('에서', 'Josa'),
 ('제', 'Noun'),
 ('주도', 'Noun'),
 ('하', 'Noun'),
 ('에', 'Josa'),
 ('가족', 'Noun'),

In [59]:
content_phrase = []

for phrase in clensing_content_data : 
    phrase = twitter.phrases(phrase)
    content_phrase.append(phrase)

print(len(content_phrase[0]))    
content_phrase[0]

364


['누구',
 '특별한 날',
 '고민',
 '고민할 것',
 '나 어른',
 '모시',
 '어딘가',
 '가야',
 '가야 하는 경우',
 '가야 하는 경우 더',
 '부모님',
 '불구',
 '어버이날',
 '어버이날 무엇',
 '준비',
 '쉽사리',
 '쉽사리 결정',
 '쉽사리 결정 내리지',
 '쉽사리 결정 내리지 못',
 '이번',
 '구로디지털단지역',
 '구로디지털단지역 맛집',
 '제 주도',
 '제 주도 하',
 '가족',
 '식사',
 '뿌듯한 미리',
 '뿌듯한 미리 어버이날기념',
 '저희',
 '저희와 같은 생각',
 '외식',
 '미리',
 '미리 예약',
 '신의',
 '신의 한 수',
 '평이',
 '평이 좋은 여느',
 '평이 좋은 여느 음식점',
 '보통',
 '여기',
 '식사 시간대',
 '손님',
 '꽤 몰리는 모습',
 '복잡한 분위기',
 '부모님 모시',
 '가기',
 '메뉴',
 '프리미엄',
 '프리미엄 등급',
 '프리미엄 등급 토마호크',
 '프리미엄 등급 토마호크와한돈',
 '프리미엄 등급 토마호크와한돈 꽃목살',
 '프리미엄 등급 토마호크와한돈 꽃목살 한돈',
 '프리미엄 등급 토마호크와한돈 꽃목살 한돈 숄더랙',
 '우대갈비',
 '아무',
 '아무 곳',
 '수 없는 귀한 부위',
 '수 없는 귀한 부위 위주',
 '방문',
 '좋은 곳',
 '는걸',
 '선택',
 '주문',
 '어려운 부위',
 '기본',
 '기본 반찬',
 '테이블',
 '테이블 위로하나',
 '테이블 위로하나 둘',
 '시작',
 '이건',
 '이건 무',
 '양념',
 '고기',
 '느끼함을 잡아줄 때',
 '느끼함을 잡아줄 때큰 역할',
 '젓갈',
 '고기랑',
 '식당',
 '식사할 때',
 '이날',
 '마찬가지',
 '싸먹을 때',
 '약간',
 '약간 흐물거리',
 '게 나오는 곳도',
 '기본찬',
 '기본찬 수준',
 '쫄깃',
 '감칠맛',
 '고기 좀',
 '이런 분이면 아실 거',
 '얼마나',
 '얼마나